# ElasticNet Regresyon
* Amaç hata  kareler toplamını minimize eden katsayıları bu katsayılara bir ceza uygulayarak bulmaktır.
* ElasticNet L1 ve L2 yaklaşımlarını birleştirir.
* Zou & Hastiie 2005
-----------------------------------------
$ SSE_{net} = \Sigma(y_i - \hat{y}_{ij})^2 + \lambda_1 \Sigma{\beta_j^2}+ \lambda_2 \Sigma{|\beta_j|}$

##### ElasticNet Regresyon Model ve Tahmini
* Gerekli küt

In [19]:
import numpy as np
import pandas as pd
from sklearn.linear_model import Ridge,Lasso,ElasticNet
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn import model_selection
import matplotlib.pyplot as plt
from sklearn.linear_model import RidgeCV, LassoCV, ElasticNetCV

In [20]:
df = pd.read_csv('Hitters.csv')
df = df.dropna()
y= df['Salary']
dms = pd.get_dummies(df[['League', 'Division','NewLeague']])
X_= df.drop(['Salary', 'League', 'Division', 'NewLeague'], axis=1)
X= pd.concat([X_,dms[['League_N','Division_W','NewLeague_N']]], axis=1)
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.25,random_state=42)

In [21]:
enet_model = ElasticNet().fit(X_train,y_train)

C:\Users\ycanf\anaconda3\Lib\site-packages\sklearn\linear_model\_coordinate_descent.py:628: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 8.488e+06, tolerance: 3.899e+03
  model = cd_fast.enet_coordinate_descent(


In [22]:
enet_model.coef_ # katsayılar

array([ -1.86256172,   8.70489065,   5.10426375,  -2.89875799,
        -1.28642985,   5.24343682,   6.04480276,  -0.14701495,
        -0.21566628,  -0.7897201 ,   1.80813117,   0.80914508,
        -0.61262382,   0.26816203,   0.27172387,  -0.36530729,
        19.2186222 , -31.16586592,   8.98369938])

In [23]:
enet_model.intercept_ # sabit terimi

-6.465955602114036

# tahmin

In [24]:
enet_model.predict(X_train) [0:50]

array([ 325.74706292,  776.06632333,  522.86508419,  107.64091955,
        449.03139566,  997.76095723,   99.78828622,  311.33763086,
        418.50335021,  879.9502608 , 1628.05423879,  831.63172575,
        909.34196881,  715.67274292,  601.33595953,  657.40417507,
       1068.2110763 ,  149.6849625 ,  190.40513329,  385.31235163,
        752.37991572, 1022.76166475,  486.94874949,  349.7945189 ,
         69.28225147,  783.98489255,  551.11613877,  205.84644387,
        367.25234577,  303.22135065,   98.44933333,  533.19866378,
       1000.16419322,  245.20490159,  448.10920305,  401.93188524,
        457.71559572,  713.00914559,  333.18157434,  235.45584771,
        210.52615243,  309.20890759,  190.6560382 ,  183.01443111,
        238.19688018, 1080.44877923,  380.19569305,  551.45922356,
        278.3820838 ,  338.53829531])

In [25]:
y_pred = enet_model.predict(X_test)

In [26]:
np.sqrt(mean_squared_error(y_test,y_pred)) ## ilkel RMSE

357.16765481812484

In [27]:
r2_score(y_test,y_pred)

0.4107022246932679

#### Model Tuning

In [28]:
enet_cv_model = ElasticNetCV(cv=10).fit(X_train,y_train)

In [30]:
enet_cv_model.alpha_

5230.764736479864

In [31]:
enet_cv_model.intercept_

-38.519405583943126

In [32]:
enet_cv_model.coef_

array([ 0.62845434,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        , -0.        ,  0.        ,  0.09788752,  0.        ,
        0.27265769,  0.19270075,  0.00758665,  0.3106529 ,  0.        ,
       -0.        ,  0.        , -0.        ,  0.        ])

#### final modeli

In [34]:
enet_tuned = ElasticNet(alpha=enet_cv_model.alpha_).fit(X_train, y_train)

In [35]:
y_pred = enet_tuned.predict(X_test)

In [36]:
np.sqrt(mean_squared_error(y_test,y_pred))

394.15280563218795

In [37]:
?ElasticNet

Init signature:
ElasticNet(
    alpha=1.0,
    *,
    l1_ratio=0.5,
    fit_intercept=True,
    precompute=False,
    max_iter=1000,
    copy_X=True,
    tol=0.0001,
    warm_start=False,
    positive=False,
    random_state=None,
    selection='cyclic',
)
Docstring:     
Linear regression with combined L1 and L2 priors as regularizer.

Minimizes the objective function::

        1 / (2 * n_samples) * ||y - Xw||^2_2
        + alpha * l1_ratio * ||w||_1
        + 0.5 * alpha * (1 - l1_ratio) * ||w||^2_2

If you are interested in controlling the L1 and L2 penalty
separately, keep in mind that this is equivalent to::

        a * ||w||_1 + 0.5 * b * ||w||_2^2

where::

        alpha = a + b and l1_ratio = a / (a + b)

The parameter l1_ratio corresponds to alpha in the glmnet R package while
alpha corresponds to the lambda parameter in glmnet. Specifically, l1_ratio
= 1 is the lasso penalty. Currently, l1_ratio <= 0.01 is not reliable,
unless you supply your own sequence of alpha.

Read mo